In [1]:
# importing libraries
import pandas as pd
import numpy as np
pd.set_option('max_colwidth', 400)
from pytz import timezone
from datetime import datetime

In [2]:
# load csv and print head
electric_data_df = pd.read_csv('Resources\cleaned_elec_data.csv')
electric_data_df.head(-20)

,Demand,Date/Time,City
0,7269.0,7/1/2018 1:00,nyc
1,690707.0,7/1/2018 2:00,nyc
2,60725.0,7/1/2018 3:00,nyc
3,6539.0,7/1/2018 4:00,nyc
4,6415.0,7/1/2018 5:00,nyc
...,...,...,...
164183,866.0,4/25/2020 0:00,seattle
164184,NaN,4/25/2020 1:00,seattle
164185,NaN,4/25/2020 2:00,seattle
164186,NaN,4/25/2020 3:00,seattle


In [3]:
#format date/time
electric_data_df["Date/Time"]=pd.to_datetime(electric_data_df["Date/Time"]).dt.strftime("%Y-%m-%d %H:%M:%S")
electric_data_df.head(-20)

,Demand,Date/Time,City
0,7269.0,2018-07-01 01:00:00,nyc
1,690707.0,2018-07-01 02:00:00,nyc
2,60725.0,2018-07-01 03:00:00,nyc
3,6539.0,2018-07-01 04:00:00,nyc
4,6415.0,2018-07-01 05:00:00,nyc
...,...,...,...
164183,866.0,2020-04-25 00:00:00,seattle
164184,NaN,2020-04-25 01:00:00,seattle
164185,NaN,2020-04-25 02:00:00,seattle
164186,NaN,2020-04-25 03:00:00,seattle


In [4]:
# define local time zones for each city
city_timezones = {
    'nyc': 'America/New_York',
    'la': 'America/Los_Angeles',
    'dallas': 'America/Chicago',
    'houston': 'America/Chicago',
    'philadelphia': 'America/New_York',
    'phoenix': 'America/Phoenix',
    'san antonio': 'America/Chicago',
    'san diego': 'America/Los_Angeles',
    'san jose': 'America/Los_Angeles',
    'seattle': 'America/Los_Angeles'
}

In [5]:
#convert datetime to Central Time
def convert_to_central(dt_str, city):
    local_tz = timezone(city_timezones[city])
    dt = datetime.strptime(dt_str, '%Y-%m-%d %H:%M:%S')
    dt_local = local_tz.localize(dt)
    dt_central = dt_local.astimezone(timezone('America/Chicago'))
    return dt_central

In [6]:
#convert 'Date/Time' column to Central Time
electric_data_df['Central Time'] = electric_data_df.apply(lambda row: convert_to_central(row['Date/Time'], row['City']), axis=1)
electric_data_df.head(-20)

,Demand,Date/Time,City,Central Time
0,7269.0,2018-07-01 01:00:00,nyc,2018-07-01 00:00:00-05:00
1,690707.0,2018-07-01 02:00:00,nyc,2018-07-01 01:00:00-05:00
2,60725.0,2018-07-01 03:00:00,nyc,2018-07-01 02:00:00-05:00
3,6539.0,2018-07-01 04:00:00,nyc,2018-07-01 03:00:00-05:00
4,6415.0,2018-07-01 05:00:00,nyc,2018-07-01 04:00:00-05:00
...,...,...,...,...
164183,866.0,2020-04-25 00:00:00,seattle,2020-04-25 02:00:00-05:00
164184,NaN,2020-04-25 01:00:00,seattle,2020-04-25 03:00:00-05:00
164185,NaN,2020-04-25 02:00:00,seattle,2020-04-25 04:00:00-05:00
164186,NaN,2020-04-25 03:00:00,seattle,2020-04-25 05:00:00-05:00


In [7]:
#convert 'Date/Time' column to Unix time
electric_data_df['Unix Time'] = pd.to_datetime(electric_data_df['Central Time']).astype('int64') // 10**9
electric_data_df.head(-20)

,Demand,Date/Time,City,Central Time,Unix Time
0,7269.0,2018-07-01 01:00:00,nyc,2018-07-01 00:00:00-05:00,1530421200
1,690707.0,2018-07-01 02:00:00,nyc,2018-07-01 01:00:00-05:00,1530424800
2,60725.0,2018-07-01 03:00:00,nyc,2018-07-01 02:00:00-05:00,1530428400
3,6539.0,2018-07-01 04:00:00,nyc,2018-07-01 03:00:00-05:00,1530432000
4,6415.0,2018-07-01 05:00:00,nyc,2018-07-01 04:00:00-05:00,1530435600
...,...,...,...,...,...
164183,866.0,2020-04-25 00:00:00,seattle,2020-04-25 02:00:00-05:00,1587798000
164184,NaN,2020-04-25 01:00:00,seattle,2020-04-25 03:00:00-05:00,1587801600
164185,NaN,2020-04-25 02:00:00,seattle,2020-04-25 04:00:00-05:00,1587805200
164186,NaN,2020-04-25 03:00:00,seattle,2020-04-25 05:00:00-05:00,1587808800


In [8]:
# drop central time column
electric_data_df.drop(columns=['Central Time'], inplace=True)
electric_data_df.head(-20)

,Demand,Date/Time,City,Unix Time
0,7269.0,2018-07-01 01:00:00,nyc,1530421200
1,690707.0,2018-07-01 02:00:00,nyc,1530424800
2,60725.0,2018-07-01 03:00:00,nyc,1530428400
3,6539.0,2018-07-01 04:00:00,nyc,1530432000
4,6415.0,2018-07-01 05:00:00,nyc,1530435600
...,...,...,...,...
164183,866.0,2020-04-25 00:00:00,seattle,1587798000
164184,NaN,2020-04-25 01:00:00,seattle,1587801600
164185,NaN,2020-04-25 02:00:00,seattle,1587805200
164186,NaN,2020-04-25 03:00:00,seattle,1587808800


In [9]:
# rename columns
electric_data_df.rename(columns={"Demand": "demand", "Date/Time": "date_time", "City": "city", "Unix Time": "unix_time"}, inplace=True)
electric_data_df.head(-20)

,demand,date_time,city,unix_time
0,7269.0,2018-07-01 01:00:00,nyc,1530421200
1,690707.0,2018-07-01 02:00:00,nyc,1530424800
2,60725.0,2018-07-01 03:00:00,nyc,1530428400
3,6539.0,2018-07-01 04:00:00,nyc,1530432000
4,6415.0,2018-07-01 05:00:00,nyc,1530435600
...,...,...,...,...
164183,866.0,2020-04-25 00:00:00,seattle,1587798000
164184,NaN,2020-04-25 01:00:00,seattle,1587801600
164185,NaN,2020-04-25 02:00:00,seattle,1587805200
164186,NaN,2020-04-25 03:00:00,seattle,1587808800


In [10]:
# show columns
electric_data_df.columns

Index(['demand', 'date_time', 'city', 'unix_time'], dtype='object')

In [11]:
# create nyc data frame
nyc_df = electric_data_df[electric_data_df["city"]=="nyc"]
nyc_df.reset_index(drop=True)

,demand,date_time,city,unix_time
0,7269.0,2018-07-01 01:00:00,nyc,1530421200
1,690707.0,2018-07-01 02:00:00,nyc,1530424800
2,60725.0,2018-07-01 03:00:00,nyc,1530428400
3,6539.0,2018-07-01 04:00:00,nyc,1530432000
4,6415.0,2018-07-01 05:00:00,nyc,1530435600
...,...,...,...,...
16531,46074.0,2020-05-19 20:00:00,nyc,1589932800
16532,40708.0,2020-05-19 21:00:00,nyc,1589936400
16533,46107.0,2020-05-19 22:00:00,nyc,1589940000
16534,4440.0,2020-05-19 23:00:00,nyc,1589943600


In [12]:
# create la data frame
la_df = electric_data_df[electric_data_df["city"]=="la"]
la_df.reset_index(drop=True)

,demand,date_time,city,unix_time
0,10681.0,2018-07-01 01:00:00,la,1530432000
1,101907.0,2018-07-01 02:00:00,la,1530435600
2,907076.0,2018-07-01 03:00:00,la,1530439200
3,9508.0,2018-07-01 04:00:00,la,1530442800
4,9431.0,2018-07-01 05:00:00,la,1530446400
...,...,...,...,...
16531,10893.0,2020-05-19 20:00:00,la,1589943600
16532,11263.0,2020-05-19 21:00:00,la,1589947200
16533,10952.0,2020-05-19 22:00:00,la,1589950800
16534,10338.0,2020-05-19 23:00:00,la,1589954400


In [13]:
# create dallas data frame
dallas_df = electric_data_df[electric_data_df["city"]=="dallas"]
dallas_df.reset_index(drop=True)

,demand,date_time,city,unix_time
0,NaN,2018-07-01 01:00:00,dallas,1530424800
1,NaN,2018-07-01 02:00:00,dallas,1530428400
2,NaN,2018-07-01 03:00:00,dallas,1530432000
3,NaN,2018-07-01 04:00:00,dallas,1530435600
4,NaN,2018-07-01 05:00:00,dallas,1530439200
...,...,...,...,...
16531,NaN,2020-05-19 20:00:00,dallas,1589936400
16532,NaN,2020-05-19 21:00:00,dallas,1589940000
16533,NaN,2020-05-19 22:00:00,dallas,1589943600
16534,NaN,2020-05-19 23:00:00,dallas,1589947200


In [14]:
# create houston data frame
houston_df = electric_data_df[electric_data_df["city"]=="houston"]
houston_df.reset_index(drop=True)

,demand,date_time,city,unix_time
0,NaN,2018-07-01 01:00:00,houston,1530424800
1,NaN,2018-07-01 02:00:00,houston,1530428400
2,NaN,2018-07-01 03:00:00,houston,1530432000
3,NaN,2018-07-01 04:00:00,houston,1530435600
4,NaN,2018-07-01 05:00:00,houston,1530439200
...,...,...,...,...
16531,NaN,2020-05-19 20:00:00,houston,1589936400
16532,NaN,2020-05-19 21:00:00,houston,1589940000
16533,NaN,2020-05-19 22:00:00,houston,1589943600
16534,NaN,2020-05-19 23:00:00,houston,1589947200


In [15]:
# create phoenix data frame
phoenix_df = electric_data_df[electric_data_df["city"]=="phoenix"]
phoenix_df.reset_index(drop=True)

,demand,date_time,city,unix_time
0,34907.0,2018-07-01 01:00:00,phoenix,1530432000
1,3256.0,2018-07-01 02:00:00,phoenix,1530435600
2,3065.0,2018-07-01 03:00:00,phoenix,1530439200
3,2929.0,2018-07-01 04:00:00,phoenix,1530442800
4,2833.0,2018-07-01 05:00:00,phoenix,1530446400
...,...,...,...,...
15955,NaN,2020-04-25 20:00:00,phoenix,1587870000
15956,NaN,2020-04-25 21:00:00,phoenix,1587873600
15957,NaN,2020-04-25 22:00:00,phoenix,1587877200
15958,NaN,2020-04-25 23:00:00,phoenix,1587880800


In [16]:
# create philadelphia data frame
philadelphia_df = electric_data_df[electric_data_df["city"]=="philadelphia"]
philadelphia_df.reset_index(drop=True)

,demand,date_time,city,unix_time
0,43907.0,2018-07-01 01:00:00,philadelphia,1530421200
1,4423.0,2018-07-01 02:00:00,philadelphia,1530424800
2,40743.0,2018-07-01 03:00:00,philadelphia,1530428400
3,5230.0,2018-07-01 04:00:00,philadelphia,1530432000
4,50752.0,2018-07-01 05:00:00,philadelphia,1530435600
...,...,...,...,...
16531,NaN,2020-05-19 20:00:00,philadelphia,1589932800
16532,NaN,2020-05-19 21:00:00,philadelphia,1589936400
16533,NaN,2020-05-19 22:00:00,philadelphia,1589940000
16534,NaN,2020-05-19 23:00:00,philadelphia,1589943600


In [17]:
# create san antonio data frame
san_antonio_df = electric_data_df[electric_data_df["city"]=="san antonio"]
san_antonio_df.reset_index(drop=True)

,demand,date_time,city,unix_time
0,NaN,2018-07-01 01:00:00,san antonio,1530424800
1,NaN,2018-07-01 02:00:00,san antonio,1530428400
2,NaN,2018-07-01 03:00:00,san antonio,1530432000
3,NaN,2018-07-01 04:00:00,san antonio,1530435600
4,NaN,2018-07-01 05:00:00,san antonio,1530439200
...,...,...,...,...
16531,NaN,2020-05-19 20:00:00,san antonio,1589936400
16532,NaN,2020-05-19 21:00:00,san antonio,1589940000
16533,NaN,2020-05-19 22:00:00,san antonio,1589943600
16534,NaN,2020-05-19 23:00:00,san antonio,1589947200


In [18]:
# create san diego data frame
san_diego_df = electric_data_df[electric_data_df["city"]=="san diego"]
san_diego_df.reset_index(drop=True)

,demand,date_time,city,unix_time
0,2023.0,2018-07-01 01:00:00,san diego,1530432000
1,1896.0,2018-07-01 02:00:00,san diego,1530435600
2,18507.0,2018-07-01 03:00:00,san diego,1530439200
3,1825.0,2018-07-01 04:00:00,san diego,1530442800
4,10798.0,2018-07-01 05:00:00,san diego,1530446400
...,...,...,...,...
16531,2220.0,2020-05-19 20:00:00,san diego,1589943600
16532,23107.0,2020-05-19 21:00:00,san diego,1589947200
16533,22207.0,2020-05-19 22:00:00,san diego,1589950800
16534,2056.0,2020-05-19 23:00:00,san diego,1589954400


In [19]:
# create san jose data frame
san_jose_df = electric_data_df[electric_data_df["city"]=="san jose"]
san_jose_df.reset_index(drop=True)

,demand,date_time,city,unix_time
0,12522.0,2018-07-01 01:00:00,san jose,1530432000
1,110745.0,2018-07-01 02:00:00,san jose,1530435600
2,11200.0,2018-07-01 03:00:00,san jose,1530439200
3,10822.0,2018-07-01 04:00:00,san jose,1530442800
4,10644.0,2018-07-01 05:00:00,san jose,1530446400
...,...,...,...,...
16531,115078.0,2020-05-19 20:00:00,san jose,1589943600
16532,110782.0,2020-05-19 21:00:00,san jose,1589947200
16533,11592.0,2020-05-19 22:00:00,san jose,1589950800
16534,11083.0,2020-05-19 23:00:00,san jose,1589954400


In [20]:
# create seattle data frame
seattle_df = electric_data_df[electric_data_df["city"]=="seattle"]
seattle_df.reset_index(drop=True)

,demand,date_time,city,unix_time
0,809.0,2018-07-01 01:00:00,seattle,1530432000
1,7079.0,2018-07-01 02:00:00,seattle,1530435600
2,753.0,2018-07-01 03:00:00,seattle,1530439200
3,748.0,2018-07-01 04:00:00,seattle,1530442800
4,745.0,2018-07-01 05:00:00,seattle,1530446400
...,...,...,...,...
15955,NaN,2020-04-25 20:00:00,seattle,1587870000
15956,NaN,2020-04-25 21:00:00,seattle,1587873600
15957,NaN,2020-04-25 22:00:00,seattle,1587877200
15958,NaN,2020-04-25 23:00:00,seattle,1587880800


In [21]:
# dst date/times
dates_to_remove = ['2018-03-11 01:00:00', '2018-11-04 01:00:00', '2019-03-10 01:00:00', '2019-11-03 01:00:00', '2020-03-08 01:00:00']

In [22]:
# drop dst dates to remove from cities
nyc_df = nyc_df[~nyc_df['date_time'].isin(dates_to_remove)].reset_index(drop=True)
la_df = la_df[~la_df['date_time'].isin(dates_to_remove)].reset_index(drop=True)
dallas_df = dallas_df[~dallas_df['date_time'].isin(dates_to_remove)].reset_index(drop=True)
houston_df = houston_df[~houston_df['date_time'].isin(dates_to_remove)].reset_index(drop=True)
philadelphia_df = philadelphia_df[~philadelphia_df['date_time'].isin(dates_to_remove)].reset_index(drop=True)
san_antonio_df = san_antonio_df[~san_antonio_df['date_time'].isin(dates_to_remove)].reset_index(drop=True)
san_deigo_df = san_diego_df[~san_diego_df['date_time'].isin(dates_to_remove)].reset_index(drop=True)
san_jose_df = san_jose_df[~san_jose_df['date_time'].isin(dates_to_remove)].reset_index(drop=True)
seattle_df = seattle_df[~seattle_df['date_time'].isin(dates_to_remove)].reset_index(drop=True)

In [23]:
# dedup
nyc_df = nyc_df.drop_duplicates(subset=['date_time']).reset_index(drop=True)
la_df = la_df.drop_duplicates(subset=['date_time']).reset_index(drop=True)
dallas_df = dallas_df.drop_duplicates(subset=['date_time']).reset_index(drop=True)
houston_df = houston_df.drop_duplicates(subset=['date_time']).reset_index(drop=True)
philadelphia_df = philadelphia_df.drop_duplicates(subset=['date_time']).reset_index(drop=True)
phoenix_df = phoenix_df.drop_duplicates(subset=['date_time']).reset_index(drop=True)
san_antonio_df = san_antonio_df.drop_duplicates(subset=['date_time']).reset_index(drop=True)
san_deigo_df = san_deigo_df.drop_duplicates(subset=['date_time']).reset_index(drop=True)
san_jose_df = san_jose_df.drop_duplicates(subset=['date_time']).reset_index(drop=True)
seattle_df = seattle_df.drop_duplicates(subset=['date_time']).reset_index(drop=True)

In [24]:
# first five to csv
nyc_df.to_csv("Output/nyc.csv", index=False)
la_df.to_csv("Output/la.csv", index=False)
dallas_df.to_csv("Output/dallas.csv", index=False)
houston_df.to_csv("Output/houston.csv", index=False)
phoenix_df.to_csv("Output/phoenix.csv", index=False)


In [25]:
# last five to csv
philadelphia_df.to_csv("Output/philadelphia.csv", index=False)
san_antonio_df.to_csv("Output/san_antonio.csv", index=False)
san_diego_df.to_csv("Output/san_diego.csv", index=False)
san_jose_df.to_csv("Output/san_jose.csv", index=False)
seattle_df.to_csv("Output/seattle.csv", index=False)

In [26]:
#convert json files to CSV for SQL
dallas_climate_df = pd.read_json('Resources/dallas.json')
dallas_climate_df.to_csv("Output/dallas_climate.csv", index=False)

In [27]:
houston_climate_df = pd.read_json('Resources/houston.json')
houston_climate_df.to_csv("Output/houston_climate.csv", index=False)

In [28]:

la_climate_df = pd.read_json('Resources/la.json')
la_climate_df.to_csv("Output/la_climate.csv", index=False)

In [29]:
nyc_climate_df = pd.read_json('Resources/nyc.json')
nyc_climate_df.to_csv("Output/nyc_climate.csv", index=False)

In [30]:
philadelphia_climate_df = pd.read_json('Resources/philadelphia.json')
philadelphia_climate_df.to_csv("Output/philadelphia_climate.csv", index=False)

In [31]:
phoenix_climate_df = pd.read_json('Resources/phoenix.json')
phoenix_climate_df.to_csv("Output/phoenix_climate.csv", index=False)

In [32]:
san_antonio_climate_df = pd.read_json('Resources/san_antonio.json')
san_antonio_climate_df.to_csv("Output/san_antonio_climate.csv", index=False)

In [33]:
san_deigo_climate_df = pd.read_json('Resources/san_diego.json')
san_deigo_climate_df.to_csv("Output/san_diego_climate.csv", index=False)

In [34]:
san_jose_climate_df = pd.read_json('Resources/san_jose.json')
san_jose_climate_df.to_csv("Output/san_jose_climate.csv", index=False)

In [35]:
seattle_climate_df = pd.read_json('Resources/seattle.json')
seattle_climate_df.to_csv("Output/seattle_climate.csv", index=False)